# Gradio GeoLlama App
This notebook shows how to launch the Gradio based GeoLlama App. We will include the full pipeline, from translation to toponym extraction to geoparsing. This is essentially the contents of `app.py`, which can be run from the CLI with `$ python geo_llama/app.py`.

In [6]:
import random
import sys
sys.path.append('..')
# third party imports
import gradio as gr
from geopy.distance import distance
from geopy.geocoders import Nominatim
# local imports
from geo_llama.translator import Translator
from geo_llama.model import RAGModel, TopoModel
from geo_llama.main import GeoLlama
from geo_llama.plotting import plot_map

In [ ]:
translator = Translator(model_size='1.2B')
topo_model = TopoModel(model_name='JoeShingleton/GeoLlama_7b_toponym', 
                    prompt_path='data/prompt_templates/prompt_template.txt',
                    instruct_path='data/prompt_templates/topo_instruction.txt',
                    input_path=None,
                    config_path='data/config_files/model_config.json')

rag_model = RAGModel(model_name='JoeShingleton/GeoLlama_7b_RAG', 
                    prompt_path='data/prompt_templates/prompt_template.txt',
                    instruct_path='data/prompt_templates/rag_instruction.txt',
                    input_path='data/prompt_templates/rag_input.txt',
                    config_path='data/config_files/model_config.json')

geo_llama = GeoLlama(topo_model, rag_model)

In [ ]:
def translate(text):
    out = translator.translate(text, out_lang='en')
    return out


def translate_name(name, coordinates):
  """We can't use the translator for the name because it tends to literally
  translate rather than preserve place names. Instead, we'll look the place up
  in Nominatim and return the english name of the first match.
  """
  user_id = f'GeoLlama_{random.uniform(1000,10000)}'
  nom = Nominatim(user_agent='Geo-Llama')
  matches = nom.geocode(name, language='en', exactly_one=False)
  # get the match which is closest to the provided coordinates
  best = matches[0]
  best_d = distance((best.latitude, best.longitude), coordinates)
  for m in matches:
    d = distance((m.latitude, m.longitude), coordinates)
    if d < best_d:
      best = m
      best_d = d
  try:
    return best.address.split(',')[0]
  except IndexError as e:
    return name + ' (unable to translate place name)'


def geoparse(text:str, translation_option='With Translation'):
    if translation_option=='With Translation':
        translated_text = translate(text)
        processed_text = translated_text['translation']
    else:
        processed_text = text


    locations = geo_llama.geoparse(processed_text)
    locations_str = ', '.join([x['name'] for x in locations])
    # Create an HTML string with highlighted place names and tooltips
    translate_cache = {}
    for loc in locations:
        lat, lon = loc['latitude'], loc['longitude']
        # if the text has been translated, we don't need to translate the name
        if translation_option == 'With translation':
            name = loc['name']
        # if with_translation is false, we still want toponyms translated.
        elif loc['name'] in translate_cache.keys():
            name = translate_cache[loc['name']]
        else:
            name = translate_name(loc['name'], (lat, lon))
            translate_cache.update({loc['name']:name})
        # Creating a tooltip for the place name with coordinates
        tooltip_html = f'<span style="background-color: yellow;" title="Toponym: {name} \n Coordinates: ({lat}, {lon})">{loc["name"]}</span>'
        processed_text = processed_text.replace(loc['name'], tooltip_html)

    # Generate the map plot
    mapped = plot_map(locations, translate_cache)

    return processed_text, locations_str, mapped

In [ ]:
def main():
    with open('data/config_files/app_info.txt', 'r') as f:
        app_info = f.read()

    input_text = gr.Textbox(label='Text')
    input_options = gr.Radio(
        label="Geoparse Mode",
        choices=["With Translation", "Without Translation"],
        value="With Translation",  # Default option
    )
    output1 = gr.Markdown()
    output2 = gr.Textbox(label='Toponyms')
    output3 = gr.Plot(label='Mapped Locations')
    demo = gr.Interface(fn=geoparse, inputs=[input_text, input_options], outputs=[output1, output2, output3], description=app_info)
    return demo

In [ ]:
demo = main()
demo.launch(share=True)